In [14]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
import re
import string
import dagshub
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [16]:
base_dir = Path(os.getenv('BASE_DIR'))
base_data_dir = Path(os.getenv('BASE_DATA_DIR'))
external_data_path = os.path.join(base_data_dir,"external")
raw_data_path = os.path.join(base_data_dir, "raw")
notebooks_path = os.path.join(base_dir, "notebooks")
os.makedirs(raw_data_path, exist_ok=True)
os.makedirs(external_data_path, exist_ok=True)

In [17]:
mlflow.set_tracking_uri('https://dagshub.com/insculptor/mlops-mini-project.mlflow')
dagshub.init(repo_owner='insculptor', repo_name='mlops-mini-project', mlflow=True)

Initialized MLflow to track repo "insculptor/mlops-mini-project"

Repository insculptor/mlops-mini-project initialized!

In [18]:
df = pd.read_csv(os.path.join(external_data_path, 'tweet_emotions.csv')).drop(columns=['tweet_id'])
df.head(5)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [19]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [20]:
df = normalize_text(df)
df.head(5)

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [21]:
x = df['sentiment'].isin(['sadness', 'happiness'])
df = df[x]
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\erdrr\AppData\Local\Temp\ipykernel_15700\2264095480.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday


In [22]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
# Set the experiment name
mlflow.set_experiment("MLops-Mini-Project_Exp3_LR-TFIDF_Hyperparameter-Tuning")

<Experiment: artifact_location='mlflow-artifacts:/f1f3897db8804cecba5456ec21a9ffd6', creation_time=1723401175201, experiment_id='3', last_update_time=1723401175201, lifecycle_stage='active', name='MLops-Mini-Project_Exp3_LR-TFIDF_Hyperparameter-Tuning', tags={}>

In [26]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [100, 200, 300, 400, 500],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

In [27]:
# Start the parent run for hyperparameter tuning
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")
    
    
    ## Log Jupyter Notebook
    notebook_path = os.path.join(notebooks_path, "exp3_LR_BOW_HyperparameterTuning.ipynb")
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    ## Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024/08/11 14:52:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.001, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3/runs/b88532f268614c16868f6e16a26fde0e.
2024/08/11 14:52:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3.


Mean CV Score: 0.0, Std CV Score: 0.0
Accuracy: 0.5108433734939759
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


2024/08/11 14:52:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.001, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'} at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3/runs/618940e5af0e4caa9c296d8a14506f62.
2024/08/11 14:52:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3.


Mean CV Score: 0.5371222417681529, Std CV Score: 0.2685611402213311
Accuracy: 0.4891566265060241
Precision: 0.4891566265060241
Recall: 1.0
F1 Score: 0.656957928802589


2024/08/11 14:52:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.001, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3/runs/4f073520e5064af98d00817334a678b5.
2024/08/11 14:52:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3.


...

Mean CV Score: 0.7865608714065474, Std CV Score: 0.008129127471489007
Accuracy: 0.7874698795180722
Precision: 0.7764932562620424
Recall: 0.7940886699507389
F1 Score: 0.7851924013638578
Mean CV Score: 0.7599745606714509, Std CV Score: 0.007412648253781466
Accuracy: 0.7681927710843374
Precision: 0.7617647058823529
Recall: 0.7655172413793103
F1 Score: 0.7636363636363637


2024/08/11 14:58:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3/runs/cb4250b9ae07403694fb79edb7edcf08.
2024/08/11 14:58:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3.
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
2024/08/11 14:58:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'} at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3/runs/a6f7690a989b4cc39dd4c5001cec0d93.
2024/08/11 14:58:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: ht

Mean CV Score: 0.7849273896987451, Std CV Score: 0.0073256594894574615
Accuracy: 0.7865060240963856
Precision: 0.7755298651252408
Recall: 0.7931034482758621
F1 Score: 0.784218217243059


2024/08/11 14:58:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 100, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3/runs/09a50f50db3c44c3b5280da59b478b21.
2024/08/11 14:58:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/3.


Mean CV Score: 0.7547984483018105, Std CV Score: 0.0146127564630285
Accuracy: 0.7691566265060241
Precision: 0.7658730158730159
Recall: 0.7605911330049261
F1 Score: 0.7632229362333168
